In [1]:
# %pip install librosa matplotlib

  Obtaining dependency information for librosa from https://files.pythonhosted.org/packages/8c/8a/2d231b35456506b7c98b3ab9bbf07917b205fed8615d2e59e976ab497fff/librosa-0.10.2.post1-py3-none-any.whl.metadata
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/32/ad/58902b481f5a294101a53ed964d68a2c2355d55622a8e9cb09fc3f517385/matplotlib-3.9.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for audioread>=2.1.9 from https://files.pythonhosted.org/packages/57/8d/30aa32745af16af0a9a650115fbe81bde7c610ed5c21b381fca0196f3a7f/audioread-3.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for numba>=0.51.0 from https://files.pythonhosted.org/packages/79/58/cb4ac5b8f7ec64200460aef1fed88258fb872ceef504ab1f989d2ff0f684/numba-0.60.0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata
  Obtaining dependency information for soundfile>=0.12.1 from https://files.pythonhosted.or

In [1]:
import os
import numpy as np
import pandas as pd
import librosa
import matplotlib.pyplot as plt
import torch ## 저같은 경우엔 pytorch를 사용하여 학습하고 있어서 torch tensor로 저장하고자합니다
from tqdm import tqdm
import IPython.display

In [2]:
%%time
# CPU times: user 20min 35s, sys: 1min 7s, total: 21min 42s
# Wall time: 21min
import numpy as np
from PIL import Image
import cv2
import os

def augment_image(image_path, output_dir):
    original_image = Image.open(image_path)
    base_name = os.path.splitext(os.path.basename(image_path))[0]

    for angle in [0, 45, 90, 135, 180, 225, 270, 315]:
        rotated_image = original_image.rotate(angle)

        # Convert PIL Image to OpenCV format
        cv_image = cv2.cvtColor(np.array(rotated_image), cv2.COLOR_RGB2BGR)

        # Flip
        flipped_image = cv2.flip(cv_image, 1)  # 1 for horizontal flip
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_flipped.jpg"), flipped_image)

        # Median Blur
        blurred_image = cv2.medianBlur(cv_image, 5)  # 5 is the kernel size
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_blurred.jpg"), blurred_image)

        # Noise
        noise = np.random.normal(0, 25, cv_image.shape).astype(np.uint8)
        noisy_image = cv2.add(cv_image, noise)
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_noisy.jpg"), noisy_image)

        # Flip + Blur
        flipped_image = cv2.flip(cv_image, 1)  # 1 for horizontal flip
        blurred_image = cv2.medianBlur(flipped_image, 5)  # 5 is the kernel size
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_flipped_blurred.jpg"), flipped_image)

        # Flip + Noise
        flipped_image = cv2.flip(cv_image, 1)  # 1 for horizontal flip
        noise = np.random.normal(0, 25, flipped_image.shape).astype(np.uint8)
        noisy_image = cv2.add(flipped_image, noise)
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_flipped_noisy.jpg"), noisy_image)

        # Blur + Noise
        blurred_image = cv2.medianBlur(cv_image, 5)  # 5 is the kernel size
        noise = np.random.normal(0, 25, blurred_image.shape).astype(np.uint8)
        noisy_image = cv2.add(blurred_image, noise)
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_blurred_noisy.jpg"), noisy_image)

        # Flip + Blur + Noise
        flipped_image = cv2.flip(cv_image, 1)  # 1 for horizontal flip
        blurred_image = cv2.medianBlur(flipped_image, 5)  # 5 is the kernel size
        noise = np.random.normal(0, 25, blurred_image.shape).astype(np.uint8)
        noisy_image = cv2.add(blurred_image, noise)
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}_flipped_blurred_noisy.jpg"), noisy_image)

        # Save the original rotated image
        cv2.imwrite(os.path.join(output_dir, f"{base_name}_rotated_{angle}.jpg"), cv_image)

def augment_all_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir, filename)
            augment_image(input_path, output_dir)

# 사용 예시
augment_all_images('/home/data/train', '/home/data/augmented_train_45D')

# 폴더에 이미지 제작과정 중 개수 확인 명령
# ls -1 /home/data/augmented_train_45D | wc -l

CPU times: user 20min 39s, sys: 1min 5s, total: 21min 45s
Wall time: 21min 2s


In [3]:
import csv
import os

def update_csv(original_csv_path, augmented_image_dir, output_csv_path):
    # 원본 CSV 파일 읽기
    with open(original_csv_path, 'r') as f:
        reader = csv.DictReader(f)
        original_data = list(reader)

    # 새로운 데이터를 저장할 리스트
    new_data = []

    # 증강된 이미지에 대한 데이터 추가
    for row in original_data:
        original_filename = row['ID']
        target = row['target']

        # 파일 이름에서 확장자 제거
        base_name = os.path.splitext(original_filename)[0]

        for angle in [0, 45, 90, 135, 180, 225, 270, 315]:
            # 회전된 이미지
            rotated_filename = f"{base_name}_rotated_{angle}.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, rotated_filename)):
                new_data.append({
                    'ID': rotated_filename,
                    'target': target
                })

            # Flipped 이미지
            flipped_filename = f"{base_name}_rotated_{angle}_flipped.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, flipped_filename)):
                new_data.append({
                    'ID': flipped_filename,
                    'target': target
                })

            # Blurred 이미지
            blurred_filename = f"{base_name}_rotated_{angle}_blurred.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, blurred_filename)):
                new_data.append({
                    'ID': blurred_filename,
                    'target': target
                })

            # Noisy 이미지
            noisy_filename = f"{base_name}_rotated_{angle}_noisy.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, noisy_filename)):
                new_data.append({
                    'ID': noisy_filename,
                    'target': target
                })

            # Flipped Blurred 이미지
            flipped_blurred_filename = f"{base_name}_rotated_{angle}_flipped_blurred.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, flipped_blurred_filename)):
                new_data.append({
                    'ID': flipped_blurred_filename,
                    'target': target
                })
                
            # Flipped Noisy 이미지
            flipped_noisy_filename = f"{base_name}_rotated_{angle}_flipped_noisy.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, flipped_noisy_filename)):
                new_data.append({
                    'ID': flipped_noisy_filename,
                    'target': target
                })
                
            # Blurred Noisy 이미지
            blurred_noisy_filename = f"{base_name}_rotated_{angle}_blurred_noisy.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, blurred_noisy_filename)):
                new_data.append({
                    'ID': blurred_noisy_filename,
                    'target': target
                })
            
            # Flipped Blurred Noisy 이미지
            flipped_blurred_noisy_filename = f"{base_name}_rotated_{angle}_flipped_blurred_noisy.jpg"
            if os.path.exists(os.path.join(augmented_image_dir, flipped_blurred_noisy_filename)):
                new_data.append({
                    'ID': flipped_blurred_noisy_filename,
                    'target': target
                })


    # 새로운 CSV 파일 작성
    with open(output_csv_path, 'w', newline='') as f:
        fieldnames = ['ID', 'target']
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        writer.writeheader()
        for row in new_data:
            writer.writerow(row)

    print(f"Updated CSV file with only augmented data has been saved to {output_csv_path}")

# 사용 예시
update_csv('/home/data/train.csv', '/home/data/augmented_train_45D', '/home/data/train_augmented_45D.csv')

Updated CSV file with only augmented data has been saved to /home/data/train_augmented_45D.csv


In [4]:
ds = pd.read_csv('/home/data/train_augmented_45D.csv')
len(ds)

100480

In [5]:
ds.head()

,ID,target
0,002f99746285dfdd_rotated_0.jpg,16
1,002f99746285dfdd_rotated_0_flipped.jpg,16
2,002f99746285dfdd_rotated_0_blurred.jpg,16
3,002f99746285dfdd_rotated_0_noisy.jpg,16
4,002f99746285dfdd_rotated_0_flipped_blurred.jpg,16


In [ ]:
# 두 폴더(/home/data/train, /home/data/augmented_train)의 이미지 파일 합쳐서 하나의 새로운 폴더(/home/data/rotate_train)로 이동
import os
import shutil

def move_images(input_dir1, input_dir2, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir1):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir1, filename)
            output_path = os.path.join(output_dir, filename)
            shutil.copy(input_path, output_path)

    for filename in os.listdir(input_dir2):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir2, filename)
            output_path = os.path.join(output_dir, filename)
            shutil.copy(input_path, output_path)

move_images('/home/data/train', '/home/data/augmented_train_45D', '/home/train_csv/rotate_train_45D')

In [ ]:
train_path = "/home/data/augmented_train_45D"
cnt = 0
for root, dirs, files in os.walk(train_path):
    cnt += len(files)

print(f"Total number of files in {train_path}: {cnt}")

Total number of files in /home/data/augmented_train: 25120


In [ ]:
import pandas as pd

data = "/home/data/train_augmented_45D.csv"
data = pd.read_csv(data)
data.head(20)

,ID,target
0,002f99746285dfdd_rotated_0.jpg,16
1,002f99746285dfdd_rotated_0_flipped.jpg,16
2,002f99746285dfdd_rotated_0_blurred.jpg,16
3,002f99746285dfdd_rotated_0_noisy.jpg,16
4,002f99746285dfdd_rotated_90.jpg,16
5,002f99746285dfdd_rotated_90_flipped.jpg,16
6,002f99746285dfdd_rotated_90_blurred.jpg,16
7,002f99746285dfdd_rotated_90_noisy.jpg,16
8,002f99746285dfdd_rotated_180.jpg,16
9,002f99746285dfdd_rotated_180_flipped.jpg,16
